In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException
import os

In [2]:
# URL страницы с фильмами
url = 'https://kino.mail.ru/cinema/all/?page='
epoch = 0

# Настройка повторных попыток
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retries)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)

# Определяем, с какой страницы начать
json_folder = 'jsons'
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

existing_files = [f for f in os.listdir(json_folder) if f.startswith('movie_data_page') and f.endswith('.json')]
if existing_files:
    # Извлекаем номера страниц из имен файлов
    page_numbers = [int(re.search(r'page(\d+)', f).group(1)) for f in existing_files]
    start_page = max(page_numbers)
else:
    start_page = 1000

for page in range(start_page,1329):
    page_url = f'https://kino.mail.ru/cinema/all/?page={page+1}'
    try:
        # Отправляем HTTP-запрос к странице
        response = http.get(page_url)
        print("-" * 20, page + 1, "-" * 20)
        # Проверяем успешность запроса
        if response.status_code == 200:
            pages_dict = dict()
            # Создаем объект BeautifulSoup для парсинга HTML
            soup = BeautifulSoup(response.content, 'html.parser')
            # Находим тег <a>
            a_tags = soup.find_all('a', href=True, class_='link link_inline color_black link-holder link-holder_itemevent link-holder_itemevent_small')
            for a_tag in a_tags:
                href_value = a_tag["href"]
                url_movie = 'https://kino.mail.ru' + href_value
                try:
                    response_movie = http.get(url_movie)
                    if response_movie.status_code == 200:
                        soup_movie = BeautifulSoup(response_movie.content, 'html.parser')

                        titleq = soup_movie.find('meta', {'property': 'mrc__share_title'})
                        if titleq:
                            title = titleq["content"]
                            pattern = re.compile(r' - смотрите онлайн( бесплатно)?')
                            title = re.sub(pattern, '', title)
                            pars = dict()
                            print(title)
                            pars["title"] = title
                            pars["url"] = url_movie
                        else:
                            print("Title not found")

                        imgq = soup_movie.find('img', class_='picture__image picture__image_cover')
                        if imgq:
                            img = imgq["src"]
                            pars["img"] = img
                        else:
                            print("Image not found")

                        date_pattern = re.compile(r'\d{1,2} \S+ \d{4}')
                        date_matches = soup_movie.find_all(string=date_pattern)
                        if date_matches:
                            dateCreated = bytes(date_matches[0].strip(), 'utf-8').decode('utf-8')
                            pars["dateCreated"] = dateCreated
                        else:
                            print("dateCreated not found")

                        durationq = soup_movie.find('meta', itemprop='duration')
                        if durationq:
                            duration = durationq["content"]
                            pars["duration"] = duration
                        else:
                            print("duration not found")

                        genreq = soup_movie.find('meta', itemprop='genre')
                        if genreq:
                            genre = genreq["content"]
                            pars["genre"] = genre
                        else:
                            print("genre not found")

                        director_div = soup_movie.find('div', {'itemprop': 'director'})
                        if director_div:
                            meta_tag = director_div.find('meta', {'itemprop': 'name'})
                            if meta_tag:
                                director_name = meta_tag['content']
                                director_name = bytes(director_name, 'utf-8').decode('utf-8')
                                pars["director"] = director_name
                            else:
                                print('Meta tag with itemprop="name" not found within the director div.')
                        else:
                            print('Div with itemprop="director" not found.')

                        actor_divs = soup_movie.find_all('div', {'itemprop': 'actor'})
                        actors = []
                        for actor_div in actor_divs:
                            meta_tag = actor_div.find('meta', {'itemprop': 'name'})
                            if meta_tag:
                                actor_name = meta_tag['content']
                                actor_name = bytes(actor_name, 'utf-8').decode('utf-8')
                                actors.append(actor_name)
                        if len(actors) != 0:
                            pars["actors"] = actors
                        else:
                            print("Actors not found")

                        if title:
                            pages_dict[f"{title}"] = pars
                except RequestException as e:
                    print(f"Request failed for movie URL {url_movie}: {e}")
                    time.sleep(5)  # Подождем перед повторной попыткой
            with open(f'jsons/movie_data_page{page+1}.json', 'w', encoding='utf-8') as json_file:
                json.dump(pages_dict, json_file, ensure_ascii=False, indent=4)
            epoch += 1
    except RequestException as e:
        print(f"Request failed for page URL {page_url}: {e}")
        time.sleep(5)  # Подождем перед повторной попыткой

-------------------- 1001 --------------------
«Эхо Парк» (1986)
«Я — вожатый форпоста» (1986)
«Я жду тебя, кит» (1986)
dateCreated not found
«Я тебя люблю» (1986)
«26 ванных комнат» (1985)
dateCreated not found
Div with itemprop="director" not found.
«9 смертей ниндзя» (1985)
«Авиатор» (1985)
«Агнец божий» (1985)
«Алмаст» (1985)
«Алов» (1985)
dateCreated not found
«Американские молнии» (1985)
«Ангелочек-мстительница» (1985)
«Анна Каренина» (1985)
«Арабески на тему Пиросмани» (1985)
dateCreated not found
Actors not found
«Арти Шоу: Время - это всё, что у тебя есть» (1985)
«Аткинс» (1985)
Div with itemprop="director" not found.
«Бабушка Метелица» (1985)
«Бал в Савойе» (1985)
dateCreated not found
«Без конца» (1985)
«Без крыши, вне закона» (1985)
«Безумец на войне» (1985)
«Белые ночи» (1985)
«Берлинский роман» (1985)
«Беспокойное сердце» (1985)
«Биозавр» (1985)
dateCreated not found
«Блаженство» (1985)
dateCreated not found
«Будущее лето» (1985)
«Великое закрытие» (1985)
dateCreated not 

KeyboardInterrupt: 

In [3]:
# # URL страницы с фильмами
# url = 'https://kino.mail.ru/cinema/all/?page='
# epoch = 0
# for page in range(1329):
#     page_url = f'https://kino.mail.ru/cinema/all/?page={page+1}'
#     # Отправляем HTTP-запрос к странице
#     response = requests.get(page_url)
#     print("-"*20, page+1, "-"*20)
#     # Проверяем успешность запроса
#     if response.status_code == 200:
#         pages_dict = dict()
#         # Создаем объект BeautifulSoup для парсинга HTML
#         soup = BeautifulSoup(response.content, 'html.parser')
#         # Находим тег <a>
#         a_tags = soup.find_all('a', href=True, class_='link link_inline color_black link-holder link-holder_itemevent link-holder_itemevent_small')
#         # print(a_tags)
#         for a_tag in a_tags:

        
#             # Выводим результаты
#             # print(f"Href: {href_value}")
#             # print(f"Decoded Text: {encoded_text}")
#             # print("-" * 20)
#             href_value = a_tag["href"]
            
#             url_movie = 'https://kino.mail.ru' + href_value
#             # print(url_movie)
#             response_movie = requests.get(url_movie)
#             if response_movie.status_code == 200:
#                 soup_movie = BeautifulSoup(response_movie.content, 'html.parser')

#                 titleq = soup_movie.find('meta', {'property': 'mrc__share_title'})
#                 if titleq:
#                     title = titleq["content"]
                    
#                     pattern = re.compile(r' - смотрите онлайн( бесплатно)?')
#                     title = re.sub(pattern, '', title)
#                     pars = dict()
#                     print(title)
#                     pars["title"] = title
#                     pars["url"] = url_movie
#                 else:
#                     print("Title not found")
#                 imgq = soup_movie.find('img', class_='picture__image picture__image_cover')
#                 if imgq:
#                     img = imgq["src"]
#                     # print(f"Image {img}")
#                     pars["img"] = img
#                 else:
#                     print("Image not found")


#                 date_pattern = re.compile(r'\d{1,2} \S+ \d{4}')

#                 date_matches = soup_movie.find_all(string=date_pattern)
#                 if date_matches:
#                     dateCreated = bytes(date_matches[0].strip(), 'utf-8').decode('utf-8')
#                     pars["dateCreated"] = dateCreated
#                 else:
#                     print("dateCreated not found")
                    
#                 durationq = soup_movie.find('meta', itemprop='duration')
#                 if durationq:
#                     duration = durationq["content"]
#                     # print(f"duration {duration}")
#                     pars["duration"] = duration
#                 else:
#                     print("duration not found")
                    
#                 genreq = soup_movie.find('meta', itemprop='genre')
#                 if genreq:
#                     genre = genreq["content"]
#                     # print(f"genre {genre}")
#                     pars["genre"] = genre
#                 else:
#                     print("genre not found")    
                    
#                 # Находим тег <div> с атрибутом itemprop="director"
#                 director_div = soup_movie.find('div', {'itemprop': 'director'})
                
#                 # Если тег <div> найден, извлекаем информацию из связанного тега <meta>
#                 if director_div:
#                     meta_tag = director_div.find('meta', {'itemprop': 'name'})
#                     if meta_tag:
#                         director_name = meta_tag['content']
#                         # Декодируем имя, если оно закодировано в байтах
#                         director_name = bytes(director_name, 'utf-8').decode('utf-8')
#                         # print(f'Director Name: {director_name}')
#                         pars["director"] = director_name
#                     else:
#                         print('Meta tag with itemprop="name" not found within the director div.')
#                 else:
#                     print('Div with itemprop="director" not found.')
                    
#                 actor_divs = soup_movie.find_all('div', {'itemprop': 'actor'})
                
#                 # Извлекаем информацию о каждом актере
#                 actors = []
#                 for actor_div in actor_divs:
#                     meta_tag = actor_div.find('meta', {'itemprop': 'name'})
#                     if meta_tag:
#                         actor_name = meta_tag['content']
#                         # Декодируем имя, если оно закодировано в байтах
#                         actor_name = bytes(actor_name, 'utf-8').decode('utf-8')
#                         actors.append(actor_name)
#                 if len(actors) != 0:
#                     pars["actors"] = actors
#                 else:
#                     print("Actors not found")
                
#                 if title:
#                     pages_dict[f"{title}"] = pars
#         # print(pages_dict)
#         epoch+=1
#         with open(f'jsons/movie_data_page{page+1}.json', 'w', encoding='utf-8') as json_file:
#             json.dump(pages_dict, json_file, ensure_ascii=False, indent=4)

                

-------------------- 1 --------------------
«Человек-паук: Нет пути домой» (2021)
«Джентльмены» (2019)
«Выживший» (2015)
«Гнев человеческий» (2021)
«Солнцепек» (2021)
«1+1» (2011)
«Веном 2» (2021)
«Фантастические твари и где они обитают» (2016)
«Марсианин» (2015)
«Лига справедливости Зака Снайдера» (2021)
«Небо» (2021)
«Мстители: Финал» (2019)
«Достать ножи» (2019)
«Сестренка» (2019)
«Собачья жизнь 2» (2019)
«Зеленая книга» (2018)
«Мстители: Война бесконечности» (2018)
«Движение вверх» (2017)
«Тор: Рагнарек» (2017)
«До встречи с тобой» (2016)
«Мстители: Эра Альтрона» (2015)
«Форсаж 7» (2015)
«Зеленая миля» (1999)
«Побег из Шоушенка» (1994)
«Ford против Ferrari» (2019)
«Джокер» (2019)
«Богемская рапсодия» (2018)
«Время первых» (2017)
«Джон Уик 2» (2017)
«Логан» (2017)
«Последний богатырь» (2017)
«Стражи Галактики. Часть 2» (2017)
«28 панфиловцев» (2016)
«Доктор Стрэндж» (2016)
«Иллюзия обмана 2» (2016)
«Первый мститель: Противостояние» (2016)
«Битва за Севастополь» (2015)
«Век Адалин» (

ConnectionError: HTTPSConnectionPool(host='kino.mail.ru', port=443): Max retries exceeded with url: /cinema/movies/575241_shokolad/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019B99514F10>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [24]:
# # Создаем объект BeautifulSoup для парсинга HTML
# soup = BeautifulSoup(requests.get('https://kino.mail.ru/cinema/movies/884743_dzhon_uik_2/').content, 'html.parser')
# # Регулярное выражение для поиска строки вида "число месяц год"
# date_pattern = re.compile(r'\d{1,2} \S+ \d{4}')

# # Поиск всех строк, соответствующих регулярному выражению
# date_matches = soup.find_all(text=date_pattern)

# # Извлечение и декодирование найденных строк
# # dates = []
# # for match in date_matches:
# #     date_text = match.strip()
# #     # Декодируем текст, если он закодирован в байтах
# #     date_text = bytes(date_matches[0].strip(), 'utf-8').decode('utf-8')
# #     dates.append(date_text)
# if date_matches:
#     print("-"*20)

# print(date_matches[0].strip())

# # Выводим найденные даты
# # for date in dates:
# #     print(f'Date: {date}')

--------------------
9 февраля 2017


C:\Users\dashi\AppData\Local\Temp\ipykernel_13548\1240079085.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  date_matches = soup.find_all(text=date_pattern)


In [3]:
import pandas as pd

In [8]:
texts = pd.read_json('jsons/movie_data_page1001.json').T.reset_index(drop=True)

In [9]:
texts

,title,url,img,dateCreated,duration,genre,director,actors
0,«Эхо Парк» (1986),https://kino.mail.ru/cinema/movies/743295_eho_...,/img/v2/nopicture/308x462@2x.png,17 апреля 1986,PT1H28M,"драма, комедия",Роберт Дорнхельм,"[Сьюзэн Дей, Том Халс, Майкл Боуэн, Кристофер ..."
1,«Я — вожатый форпоста» (1986),https://kino.mail.ru/cinema/movies/927700_ya_v...,/img/v2/nopicture/308x462@2x.png,29 октября 1986,PT1H25M,драма,Геннадий Полока,"[Валерий Золотухин, Павел Кадочников, Василий ..."
2,"«Я жду тебя, кит» (1986)",https://kino.mail.ru/cinema/movies/926707_ya_z...,https://resizer.mail.ru/p/3dc16f77-feb0-5127-8...,NaN,PT0H10M,"короткометражный, мультфильмы",Леонид Шварцман,"[Светлана Степченко, Татьяна Курьянова, Татьян..."
3,«Я тебя люблю» (1986),https://kino.mail.ru/cinema/movies/728635_ja_t...,https://resizer.mail.ru/p/66edb65b-2696-5f7f-b...,14 мая 1986,PT1H41M,"драма, мелодрама, фэнтези",Марко Феррери,"[Кристофер Ламберт, Эдди Митчелл, Флора Барилл..."
4,«26 ванных комнат» (1985),https://kino.mail.ru/cinema/movies/728525_26_v...,/img/v2/nopicture/308x462@2x.png,NaN,PT0H26M,документальный,NaN,[Джеффри Палмер]
5,«9 смертей ниндзя» (1985),https://kino.mail.ru/cinema/movies/729328_9_sm...,/img/v2/nopicture/308x462@2x.png,10 августа 1985,PT1H34M,"драма, боевик",Эммет Олстон,"[Сё Косуги, Брент Хафф, Эмилия Кроу, Блэкки Дэ..."
6,«Авиатор» (1985),https://kino.mail.ru/cinema/movies/736776_avia...,https://resizer.mail.ru/p/178acd68-6b96-5d41-8...,1 марта 1985,PT1H36M,"драма, приключения",Джордж Миллер,"[Кристофер Рив, Розанна Аркетт, Джек Уорден, С..."
7,«Агнец божий» (1985),https://kino.mail.ru/cinema/movies/730235_agne...,https://resizer.mail.ru/p/39cea410-6232-57b8-a...,21 августа 1985,PT1H38M,"драма, триллер",Норман Джуисон,"[Джейн Фонда, Энн Бэнкрофт, Мег Тилли, Энн Пит..."
8,«Алмаст» (1985),https://kino.mail.ru/cinema/movies/814163_almast/,https://resizer.mail.ru/p/c4d6f95e-d62c-589b-b...,7 января 1986,PT0H0M,"драма, исторический",Тигран Левонян,"[Арусь Папян, Рудольф Харатян, Иван Диванян, Г..."
9,«Алов» (1985),https://kino.mail.ru/cinema/movies/804865_alov/,/img/v2/nopicture/308x462@2x.png,NaN,PT0H52M,документальный,Владимир Наумов,"[Владимир Наумов, Наталья Белохвостикова, Юрий..."


In [17]:
import pandas as pd

df = pd.read_csv('../data/movie_data.csv').drop(columns=['Unnamed: 0'])

/tmp/ipykernel_74629/3095024210.py:3: DtypeWarning: Columns (3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/movie_data.csv').drop(columns=['Unnamed: 0'])


In [18]:
df['img'].fillna('noimgtoday')

0        https://resizer.mail.ru/p/77287c77-f712-5110-8...
1        https://resizer.mail.ru/p/269f49ca-a456-59bf-a...
2        https://resizer.mail.ru/p/e8c2b865-d24d-585b-b...
3        https://resizer.mail.ru/p/61fc89c3-5ccc-598a-9...
4        https://resizer.mail.ru/p/311e265e-628d-557b-9...
                               ...                        
52457                                           noimgtoday
52458                                           noimgtoday
52459                                           noimgtoday
52460                                           noimgtoday
52461                                           noimgtoday
Name: img, Length: 52462, dtype: object

In [19]:
df.isna().sum()

title                  0
url                    0
img                 7456
time_sec               0
genre                  0
director               0
actors              7456
description            0
year                   0
ужасы                  0
фэнтези                0
вестерн                0
семейный               0
мультфильмы            0
приключения            0
военный                0
фантастика             0
драма                  0
комедия                0
мелодрама              0
боевик                 0
триллер                0
музыкальный            0
документальный         0
короткометражный       0
криминал               0
эротика                0
детектив               0
детский                0
мюзикл                 0
аниме                  0
биография              0
спорт                  0
исторический           0
title_full             0
dtype: int64